In [2]:
import torch
import os
import glob
import matplotlib.pyplot as plt

import queue

import threading

from tqdm.notebook import tqdm

from collections import namedtuple

import time

import random

In [13]:
import cv2 
import numpy as np
from PIL import Image

In [4]:
import torch
import torch.functional as F
import torch.nn as nn

In [5]:
Batch = namedtuple('batch', ('before', 'after'))

In [6]:
emoticon_file_list = glob.glob('resource/*.png')
emoticon_file_list.sort()

In [7]:
image = cv2.imread(emoticon_file_list[-1])
image = image[:, :, (2, 1, 0)]
image = image / 2 ** 8
# plt.imshow(image)

In [8]:
# Remove white images

for file in tqdm(emoticon_file_list):
    
    image = cv2.imread(file)
    
    if np.min(image) == 255:
        print(file)
        image = image[:, :, (2, 1, 0)]
        plt.figure()
        plt.imshow(image)
        plt.show()
        os.remove(file)

In [29]:
for file in tqdm(emoticon_file_list):
    
    image = cv2.imread(file)
    
    image = image[:, :, (2, 1, 0)]
        
    if image.shape[0] != image.shape[1]:
        
        white_image = np.ones((124, 124, 3), dtype=np.uint8) * 255
        white_image[:image.shape[0], :image.shape[1], :image.shape[2]] = image
        
        plt.figure()
        plt.imshow(white_image)
        plt.show()
        print(file)
        
        im = Image.fromarray(white_image)
        im.save(file)

In [ ]:
def punch_image(image, p=0.1):
    
    bool_mask = np.random.choice((True, False), image.shape[1:3], True, (1-p, p))
    bool_mask = np.stack([bool_mask, bool_mask, bool_mask], axis=0)
    
    punched_image= np.where(bool_mask, image, np.ones(image.shape))
    
    return punched_image

In [ ]:
# x = np.rollaxis(image, 2, 0)
# x.shape
# y = np.rollaxis(x, 0, 3)
# plt.imshow(y)

In [ ]:
# Punch test

# for j in range(3):
    
#     file = np.random.choice(emoticon_file_list)
    
#     image = cv2.imread(file)
#     image = image[:, :, (2, 1, 0)]
#     image = image / 2 ** 8

#     punched_image = image

#     fig, axes = plt.subplots(4, 4, figsize=(8, 8))

#     for i in range(16):

#         axes[i//4][i%4].imshow(punched_image, aspect='auto')

#         punched_image = punch_image(punched_image, 0.15)

#     plt.tight_layout()
#     plt.show()

In [ ]:
def batch_list_to_batch(batch_list):
    try:
        before_batch = np.stack([batch_tuple[0] for batch_tuple in batch_list], axis=0)
        after_batch = np.stack([batch_tuple[1] for batch_tuple in batch_list], axis=0)
    except:
        [print(batch_tuple[0].shape) for batch_tuple in batch_list]
        [print(batch_tuple[1].shape) for batch_tuple in batch_list]
    return (before_batch, after_batch)

class PunchImageFeeder:
    
    def __init__(self, file_list):
        self.file_list = file_list
        self.queue = queue.Queue(maxsize=100)
        self.finished = False
        self.batch_size = 64
        
    def start_feeding(self):
        
        batch_tuple_list = []
        
        for file in self.file_list[::10]:
            image = cv2.imread(file)
            image = image[:, :, (2, 1, 0)]
            image = image / 2 ** 8
            image = np.rollaxis(image, 2, 0)
            
            for j in range(15):
                punched_image = punch_image(image, 0.15)
                batch_tuple_list.append(Batch(punched_image, image))
                
                if len(batch_tuple_list) >= self.batch_size:
                    random.shuffle(batch_tuple_list)
                    batch = batch_list_to_batch(batch_tuple_list)
                    self.queue.put(batch)
                    batch_tuple_list = list()
                    
                image = punched_image
                
        if len(batch_tuple_list) > 0:
            random.shuffle(batch_tuple_list)
            batch = batch_list_to_batch(batch_tuple_list)
            self.queue.put(batch)
            batch_tuple_list = list()
                
        self.finished = True
             
    def generator(self):
        
        self.finished = False
        
        t = threading.Thread(target=self.start_feeding)
        t.start()
        
        while not self.finished:
            try:
                batch = self.queue.get_nowait()
                yield batch
            except:
                time.sleep(1)
        
        t.join()

In [ ]:
punch_image_feeder = PunchImageFeeder(emoticon_file_list)

In [ ]:
# nn.functional.pad(torch.tensor(image), (0, 0, 1, 1, 1, 1)).shape

In [ ]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 16, 3, padding=1)
        self.conv4 = nn.Conv2d(16, 3, 3, padding=1)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        
        return x
        

In [ ]:
for i, batch in enumerate(punch_image_feeder.generator()):
    
    batch_before, batch_after = batch
    
    if i % 10 == 0:
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        axes[0].imshow(np.rollaxis(batch_before[0], 0, 3))
        axes[1].imshow(np.rollaxis(batch_after[0], 0, 3))
        plt.show()

In [ ]:
batch[0][0]